In [ ]:
from diagrams import Diagram, Cluster, Edge
from diagrams.onprem.client import User
from diagrams.aws.storage import S3
from diagrams.custom import Custom
from diagrams.programming.framework import FastAPI
from diagrams.onprem.container import Docker
from diagrams.gcp.compute import GCE  # Added import for GCP VM

# Paths to custom icons
streamlit_icon = "streamlit_icon.png"
openai_icon = "openai_icon.png"
docker_icon = "docker_icon.png"
fastapi_icon = "fastapi_icon.png"
airflow_icon = "airflow_icon.png"
snowflake_icon = "snowflake_icon.png"
milvus_icon = "milvus_icon.png"
selenium_icon = "selenium_icon.png"
cfa_icon = "cfa_icon.png"
llamaparse_icon = "llamaparse_icon.png"

# Diagram layout attributes
graph_attr = {
    "fontsize": "15",
    "splines": "ortho",
    "rankdir": "LR",
    "compound": "true"
}

edge_attr = {
    "color": "black",
}

# Create the main diagram
with Diagram("Multi-modal RAG system", show=True, graph_attr=graph_attr, edge_attr=edge_attr):
    # Data Ingestion Layer
    with Cluster("Data Ingestion Layer"):
        cfa_website = Custom("CFA Website", cfa_icon)
        selenium = Custom("Selenium", selenium_icon)
        airflow = Custom("Airflow", airflow_icon)
        
        cfa_website >> selenium >> airflow

    # Storage Layer
    with Cluster("Storage Layer"):
        snowflake = Custom("Snowflake", snowflake_icon)
        s3 = S3("AWS S3")
        milvus = Custom("Milvus Vector Store", milvus_icon)
            
        airflow >> [s3, snowflake]

    # Frontend Layer
    with Cluster("Frontend Layer"):
        user = User("End User")
        streamlit = Custom("Streamlit", streamlit_icon)

    # Processing & Backend Layer
    with Cluster("Processing & Backend Layer"):
        with Cluster("Docker Deployment", graph_attr={"labeljust": "r"}):
            docker_compose = Custom("Docker Compose", docker_icon)
            docker_fastapi = Custom("FastAPI Container", docker_icon)
            docker_streamlit = Custom("Streamlit Container", docker_icon)
            gcp_vm = GCE("GCP VM")  # Added GCP VM
            
            # Docker containers connect to Docker Compose, and Docker Compose connects to GCP VM
            [docker_fastapi, docker_streamlit] >> docker_compose >> gcp_vm

        fastapi = Custom("FastAPI", fastapi_icon)
        openai = Custom("OpenAI API", openai_icon)
        llama_parse = Custom("LlamaParse", llamaparse_icon)

        # Processing connections
        fastapi >> docker_fastapi

        # Bidirectional connections with arrows
        fastapi >> Edge(color="black") >> openai
        openai >> Edge(color="black") >> fastapi
        
        fastapi >> Edge(color="black") >> llama_parse
        llama_parse >> Edge(color="black") >> fastapi
        
        fastapi >> Edge(color="black") >> s3
        s3 >> Edge(color="black") >> fastapi
        
        fastapi >> Edge(color="black") >> snowflake
        snowflake >> Edge(color="black") >> fastapi
        
        fastapi >> Edge(color="black") >> milvus
        milvus >> Edge(color="black") >> fastapi

    # Frontend connections
    user >> streamlit
    streamlit >> docker_streamlit
    fastapi >> streamlit  # Direct connection outside docker
    streamlit >> Edge(color="black") >> fastapi  # Connection through docker